In [1]:
import cv2
import numpy as np
import mediapipe as mp

In [4]:

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

In [5]:
canvas = None
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 0, 0)]  # Blue, Green, Red, Eraser
color_index = 0
brush_thickness = 5
drawing_mode = True 

In [7]:
cap = cv2.VideoCapture(0)

def is_pinching(hand_landmarks, frame_width, frame_height):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

    thumb_x, thumb_y = int(thumb_tip.x * frame_width), int(thumb_tip.y * frame_height)
    index_x, index_y = int(index_tip.x * frame_width), int(index_tip.y * frame_height)

    distance = np.linalg.norm([thumb_x - index_x, thumb_y - index_y])
    return distance < 40  

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if canvas is None:
        canvas = np.zeros_like(frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            h, w, _ = frame.shape
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

            if is_pinching(hand_landmarks, w, h):
                drawing_mode = not drawing_mode

            if drawing_mode:
                if colors[color_index] == (0, 0, 0): 
                    cv2.circle(canvas, (cx, cy), brush_thickness, colors[color_index], -1)
                else:
                    cv2.circle(frame, (cx, cy), 10, colors[color_index], -1)
                    cv2.circle(canvas, (cx, cy), brush_thickness, colors[color_index], -1)

    combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)
    instructions = [
        "Press 'c' to clear canvas",
        "Press 's' to save canvas",
        "Press 'up/down' to adjust brush size",
        "Pinch gesture: Toggle drawing mode",
        "Press '1-4': Change color (1: Blue, 2: Green, 3: Red, 4: Eraser)"
    ]

    for i, line in enumerate(instructions):
        cv2.putText(combined, line, (10, 20 + i * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    cv2.imshow("Drawing Board", combined)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'): 
        canvas = None
    elif key == ord('s'):  
        cv2.imwrite("drawing.png", canvas)
        print("Canvas saved as 'drawing.png'")
    elif key == 27: 
        break
    elif key in [ord('1'), ord('2'), ord('3'), ord('4')]:
        color_index = key - ord('1')
    elif key == ord('u'):  
        brush_thickness += 1
    elif key == ord('d'): 
        brush_thickness = max(1, brush_thickness - 1)

cap.release()
cv2.destroyAllWindows()